In [1]:
using Pkg
Pkg.activate("."); 
Pkg.instantiate()

  Activating environment at `C:\Users\email\OneDrive\Dokumente\Uni\Köln\21WS\ATIS3\GroupProject\ATIES3_group_project-main\Project.toml`


In [4]:
using JuMP, Plots, CPLEX, DataFrames, XLSX # IterTools, Statistics, StatsPlots, 

┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1342


In [5]:
data = Dict()
data["time_series"] = DataFrame(XLSX.readtable("data_ATIS3.xlsx", "time_series")...)
data["scenarios"] = DataFrame(XLSX.readtable("data_ATIS3.xlsx", "scenarios")...) ;

Orientiert an C1_Producer_Pool.

\Omega = 100 Szenarien für jede betrachtete Periode t Element 1:T

Ich habe die Parameter jetzt nicht in einem Dict gespeichert, weil das das Aufrufen später weniger voluminös macht.

In [6]:
### Sets ### (or rather, set cardinalities)
# Number scenarios
Ω = 100
# Number time periods
T = 2

### Scalars ###
# Factors to FC that limit DA bids
FC_DA_fac_lo = 0.85
FC_DA_fac_up = 1.15
# Maximum power production
q_max = 100

### Vectors ###
## Length T
# Prices in DA, 15 and ID
p_DA  = data["time_series"][1:T,"p_da"]
p_15  = data["time_series"][1:T,"p_15"]
p_ID3 = data["time_series"][1:T,"p_id3"]
# Power ForeCasted and real
q_FC = ones(T)
q_r = ones(T)
## Length Ω
# FC error sign; 1st stage decission
F_sgn = vcat(-1 * ones(size(data["scenarios"].neg)), ones(size(data["scenarios"].pos)))
# Relative FC errors
q_ΔF = vcat(data["scenarios"].neg, data["scenarios"].pos)
# Non-anticipativity vector (same 1st stage -> same p_15)
A = ones(length(F_sgn)-1)
for a in 1:length(F_sgn)-1
    if F_sgn[a] != F_sgn[a+1]
        A[a] = 0
    end
end
# Scenario probablilities
pi = 1/Ω * ones(Ω)
;

In [7]:
wind = Model(CPLEX.Optimizer);

In [17]:
q_DA = @variable(wind, [1:T]) # No ω dependence due to non-anticipativity!?
q_15 = @variable(wind, [1:T, 1:Ω]);
# q_ID is in fact not needed, as it only depends on q_DA, q_15 and q_r

In [9]:
@objective(wind, Max,
    sum(
        sum(
            p_DA[t]*q_DA[t] + p_15[t]*q_15[t, ω] + p_ID3[t]*(q_r[t]-q_DA[t]-q_15[t, ω])
        for ω in 1:Ω)
    for t in 1:T)
);

In [18]:
MinMaxPowerDA = @constraint(wind, [t in 1:T],
    0 <= q_DA[t] <= q_max);

MaxPowerTot = @constraint(wind, [t in 1:T, ω in 1:Ω],
    q_DA[t]+q_15[t, ω] <= q_max);

# Non-anticipativity; gets only activated, if 1st stage of two scenarios is equal;
# q_15 has to be the same then.
nonAnticip = @constraint(wind,[t in 1:T, ω in 1:Ω-1],
    A[ω] * (q_15[t,ω]-q_15[t,ω+1]) == 0
);

In [ ]:
### OLD: Save in Dict
par = Dict(
    ### Sets (or rather, set cardinalities)
    # Number scenarios
    :Ω => 100,
    # Number time periods
    :T => 2,
    
    ### Scalars
    # Factors to FC that limit DA bids
    :FC_DA_fac_lo => 0.85,
    :FC_DA_fac_up => 1.15,
    
    ### Vectors ###
    ## Length T
    # Prices in DA, 15 and ID
    :p_DA  => data["time_series"][1:param[:T],"p_da"],
    :p_15  => data["time_series"][1:param[:T],"p_15"],
    :p_ID3 => data["time_series"][1:param[:T],"p_id3"],
    # Power ForeCasted and real
    :q_FC => ones(T)
    :q_r => ones(T)
    ## Length Ω
    # FC error sign; 1st stage decission
    :F_sgn => append!(-1 * ones(size(data["scenarios"].neg)), ones(size(data["scenarios"].pos))),
    # Relative FC errors
    :q_ΔF => append!(data["scenarios"].neg, data["scenarios"].pos),
    # Scenarios probablilities
    :pi => 1/param[:Ω] * ones(param[:Ω]),
    
    
)

In [ ]:
### OLD: Stages kriegen einzelne Nummerierungen
par = Dict(
    ### Sets (or rather, set cardinalities)
    # Number scenarios first stage
    :Ω => 2,
    # Number scenarios sec stage
    :Φ => 50,
    # Number time periods
    :T => 2,
    
    ### Scalars
    # Factors to FC that limit DA bids
    :FC_DA_fac_lo => 0.85,
    :FC_DA_fac_up => 1.15,
    
    ### Vectors
    # Prices in DA, 15 and ID
    :p_DA  => data["time_series"][1:param[:T],"p_da"],
    :p_15  => data["time_series"][1:param[:T],"p_15"],
    :p_ID3 => data["time_series"][1:param[:T],"p_id3"],
    # Scenarios probablilities
    :pi_ω => 1/param[:Ω] * ones(param[:Ω]),
    :pi_ϕ => 1/param[:Φ] * ones(param[:Φ]),
    # 1st stage decission: Forecast sign
    :FC_sgn => [-1, +1],
    
    
);